# Лаба 1

In [237]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

In [238]:
f = lambda x1, x2 : np.array([1, x1, x1**2, x2**2, x2]) 
Q = np.array([1, 1, 0.01, 10, 0.01])
# xi = [-1, 1], i = 1, 2

In [239]:
u = lambda x1, x2: x1 + 0.01 * x1**2 + 0.01 * x2 + 10*x2**2 + 1

In [240]:
#fig = plt.figure(figsize = (10, 10))
#ax = fig.add_subplot(1, 1, 1, projection = '3d')
#x1_val = np.linspace(-1, 1, 100)
#x2_val = np.linspace(-1, 1, 100)
#x1, x2 = np.meshgrid(x1_val, x2_val)
#z = u(x1, x2)
#
#surf = ax.plot_surface(x1, x2, z, rstride = 3, cstride = 3, cmap = cm.plasma)
#ax.set_xlabel('X1', labelpad = 15)
#ax.set_ylabel('X2', labelpad = 15)
#ax.set_zlabel('U', labelpad = 15)

In [241]:
#fig = plt.subplots()
#x = np.linspace(-1, 1,100)
#plt.plot(x, u(x, 0))
#plt.title('u(x1, 0)')
#plt.xlabel('x1')
#plt.ylabel('U')
#plt.show()

In [242]:
#fig = plt.subplots()
#x = np.linspace(-1, 1,100)
#plt.plot(x, u(0, x))
#plt.title('u(0, x2)')
#plt.xlabel('x2')
#plt.ylabel('U')
#plt.show()

# Генерация данных

In [243]:
import random

In [244]:
#генерим 100 точек
x = []
y = []
for i in range(100):
    x.append(random.uniform(-1,1))
    y.append(random.uniform(-1,1))
#
#fig = plt.subplots()
#
#
#plt.scatter(x, y)
#plt.show()

In [245]:
import pandas as pd
df = pd.DataFrame(columns = ["i", "x1", "x2", "u", "y","y_o","u-y_o", "y-y_o"])
for i in range(len(x)):
    df.loc[i] = [i, x[i], y[i], u(x[i], y[i]),0,0,0,0]


In [246]:
vec_u = np.array(df.u)

In [247]:
df

,i,x1,x2,u,y,y_o,u-y_o,y-y_o
0,0.0,-0.923474,-0.760933,5.867642,0.0,0.0,0.0,0.0
1,1.0,0.815479,-0.031514,1.831745,0.0,0.0,0.0,0.0
2,2.0,-0.233809,-0.123906,0.919026,0.0,0.0,0.0,0.0
3,3.0,0.864464,0.714542,6.984788,0.0,0.0,0.0,0.0
4,4.0,-0.999020,0.267175,0.727455,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
95,95.0,-0.604829,0.351013,1.634439,0.0,0.0,0.0,0.0
96,96.0,-0.225485,0.153075,1.010875,0.0,0.0,0.0,0.0
97,97.0,-0.961953,-0.661870,4.421405,0.0,0.0,0.0,0.0
98,98.0,-0.468753,0.687518,5.267129,0.0,0.0,0.0,0.0


In [248]:
p = 0.1

In [249]:
mean = df.u.mean()
w2 = np.dot(df.u - mean, df.u - mean)/(len(df.u) - 1)
sigma = p*w2
sigmaE = sigma
sigmaE

1.0390001351127587

In [250]:
w2

10.390001351127585

# Лаба 2

## Неизвестная случайная ошибка

In [251]:
import scipy
from scipy import optimize
import math

In [252]:
def l2(f, Q, u, p):
    for i in range(len(df.u)):
        df.u[i] = u(df.x1[i], df.x2[i])
    
    mean = df.u.mean()
    w2 = np.dot(df.u - mean, df.u - mean)/(len(df.u) - 1)
    sigma = p*w2
    
    e = np.random.normal(0, sigma**0.5, len(df.u))
    for i in range(len(df['u'])):
        df['y'][i] = df["u"][i] + e[i]
        
    X = []
    for i in range(len(Q)):
        X.append([])
        for j in range(len(df['u'])):
            X[i].append(f(df['x1'][j],df['x2'][j])[i])
    X = np.matrix(X)
    
    QT = np.linalg.inv(X.dot(X.T))
    QT = QT.dot(X)
    QT = QT.dot(np.matrix(df.y).T)
    
    y_o = X.T.dot(QT)
    for i in range(len(df['u'])):
        df['y_o'][i] = y_o[i]
        df['y-y_o'][i] = df.y[i]-y_o[i]
        df['u-y_o'][i] = df.u[i]-df.y_o[i]
    e_o = df.y - df.y_o
    sigma2 = (e_o.T.dot(e_o)/(len(df["u"])-len(Q)))
    F = sigma2/sigma
    print(sigma)
    return QT, sigma2, F, X 

In [253]:
l2(f,Q,u,0.1)

1.0390001351127587


(matrix([[ 1.00729921],
         [ 1.01841239],
         [ 0.16454225],
         [10.30119107],
         [ 0.10607265]]),
 1.4034782407160369,
 1.350796976136796,
 matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
           1.00000000e+00,  1.00000000e+00, 

In [254]:
#df.to_excel("output.xlsx")

# Лаба 3

#### 1) Изменить модель регрессии, добавив в неё дополнительный регрессор, ранее не вошедший в состав модели, порождающей данные. Не генерируя новых данных, найти точечные оценки всех параметров расширенной модели. В дальнейшем при рассмотрении этой расширенной модели анализе должно быть показано, что параметр при дополнительном регрессоре незначим.

In [255]:
import numpy as np

In [256]:
f_old = lambda x1, x2 : np.array([1, x1, x1**2, x2**2, x2]) 
Q_old = np.array([1, 1, 0.01, 10, 0.01])
u_old = lambda x1, x2: x1 + 0.01 * x1**2 + 0.01 * x2 + 10*x2**2 + 1

# Добавим доп регрессор x1*x2
f = lambda x1, x2 : np.array([1, x1, x1**2, x2**2, x2, x1 * x2]) 
Q = np.array([1, 1, 0.01, 10, 0.01, 0.01])
u = lambda x1, x2: x1 + 0.01 * x1**2 + 0.01 * x2 + 10*x2**2 + 0.01*x1 * x2 + 1

In [257]:
#Оценки старой модели: 
QT_old, sigma2_old, F_old, X_old = l2(f_old,Q_old, u_old, 0.1)
print(QT_old, sigma2_old, F_old)

1.0390001351127587
[[ 0.80092102]
 [ 0.82684247]
 [-0.14216552]
 [10.27123256]
 [ 0.07712779]] 1.1981971309020412 1.153221342721005


In [258]:
#Оценки новой модели:
QT, sigma2, F, X = l2(f,Q,u,0.1)
print(QT, sigma2, F)
X.shape

1.0389669355707252
[[ 1.07574857]
 [ 0.92484511]
 [-0.32782173]
 [10.17214891]
 [-0.09514903]
 [ 0.2305393 ]] 1.1006312165125576 1.059351533557667


(6, 100)

#### 2) Построить доверительные интервалы для каждого параметра модели регрессии.

In [259]:
#n=100 m=6 квантиль распределение Стьюдента = 1.9850
t=1.950

In [260]:
djj=np.diagonal(np.linalg.inv((X.dot(X.T))))
sigma_Qj = np.diagonal(sigma2*np.linalg.inv((X.dot(X.T))))**0.5

Qmin = QT-t*np.matrix(sigma_Qj).T
Qmax = QT+t*np.matrix(sigma_Qj).T
print(Qmin)
print()
print(Q)
print(QT)
print()
print(Qmax)

[[ 0.66299689]
 [ 0.55452492]
 [-1.0259542 ]
 [ 9.50848096]
 [-0.43042202]
 [-0.39189847]]

[ 1.    1.    0.01 10.    0.01  0.01]
[[ 1.07574857]
 [ 0.92484511]
 [-0.32782173]
 [10.17214891]
 [-0.09514903]
 [ 0.2305393 ]]

[[ 1.48850026]
 [ 1.29516529]
 [ 0.37031073]
 [10.83581686]
 [ 0.24012395]
 [ 0.85297707]]


In [261]:
flag = 1
for i in range(len(Q)):
    if Q[i]<Qmax[i][0] and Q[i]>Qmin[i][0] and QT[i][0]<Qmax[i][0] and QT[i][0]>Qmin[i][0]:
        flag=1
    else:
        flag=0
    print(flag)

1
1
1
1
1
1


In [262]:
Qmax.shape

(6, 1)

#### 3) Проверить гипотезу о незначимости каждого параметра модели.

In [263]:
#F(1,94) ~ 3.92 вроде бы

In [264]:
F=[]
for j in range(len(Q)):
    F.append(float(QT[j][0])**2/(sigma2*djj[j]))
    if F[j]<3.92:
        print('Принимается')
    else:
        print('Отвергается')
print(F)

Отвергается
Отвергается
Принимается
Отвергается
Принимается
Принимается
[25.829326054133087, 23.716601760977564, 0.838434297553756, 893.2909826449056, 0.30625336507964535, 0.5216356562029222]


#### 4)  Проверить гипотезу о значимости самой регрессии.

In [265]:
RSS=(np.array(df.y)-X.T.dot(QT).T)
RSS=float(RSS.dot(RSS.T))
print(RSS)
RSSh=0
for i in range(len(df.y)):
    #RSSh+=(df.y[i]-df.u[i])**2
    RSSh+=(df.y[i]-np.mean(df.u))**2
print(RSSh)
m=6
n=100
q=m-1
F=((RSSh-RSS)/q)/(RSS/(n-m))
print(F)

103.4593343521804
1167.6429582952612
193.3769655044014


In [266]:
#F=278 > F(5,94) = 2.2899
#Гипотеза о незначимости регрессии отвергается

#### 5)  Рассчитать прогнозные значения для математического ожидания функции отклика для всего интервала действия одного из факторов, зафиксировав значения других факторов на границе или в центре области их определения.

In [267]:
sigma_nu = lambda x1, x2 : sigma2*float(f(x1,x2).dot(np.linalg.inv(X.dot(X.T)).dot(np.matrix(f(x1,x2)).T)))**0.5

In [268]:
def sigma_nu (x1, x2) : 
    return sigma2*np.array(f(x1,x2)*np.linalg.inv(X.dot(X.T))*np.matrix(f(x1,x2)).T)

In [269]:
nu = lambda x1, x2, Q : sum(np.array([1*Q[0], x1*Q[1], x1**2*Q[2], x2**2*Q[3], x2*Q[4], x1 * x2*Q[5]]))
nu_min = lambda x1, x2, Q : np.float64(nu(x1,x2,Q) - t*(sigma_nu(x1,x2)[0][0]))
nu_max = lambda x1, x2, Q : np.float64(nu(x1,x2,Q) + t*(sigma_nu(x1,x2)[0][0]))

In [270]:
#fig = plt.subplots()
#x = np.linspace(-1, 1,100)
#
#plt.plot(x, nu(x, 0,np.array(QT)))
#plt.plot(x, nu_min(x, 0,np.array(QT)))
#plt.plot(x, nu_max(x, 0,np.array(QT)))
#plt.title('u(x1, 0)')
#plt.show()

In [271]:
#fig = plt.subplots()
#x = np.linspace(-1, 1,100)
#
#plt.plot(x, nu(0, x,np.array(QT)))
#plt.plot(x, nu_min(0, x,np.array(QT)))
#plt.plot(x, nu_max(0, x,np.array(QT)))
#plt.title('u(0, x2)')
#plt.show()

#### 6) Построить график прогнозных значений и доверительной полосы для математического ожидания функции отклика и для самого отклика.

In [272]:
#fig = plt.subplots()
#x = np.linspace(-0.01, 0.01,100)
#
#plt.plot(x, nu(x, 0,np.array(QT)))
#plt.plot(x, nu_min(x, 0,np.array(QT)))
#plt.plot(x, nu_max(x, 0,np.array(QT)))
#plt.title('u(x1, 0)')
#plt.show()

In [273]:
#fig = plt.subplots()
#x = np.linspace(-0.0, 0.1,100)
#
#plt.plot(x, nu(0, x,np.array(QT)))
#plt.plot(x, nu_min(0, x,np.array(QT)))
#plt.plot(x, nu_max(0, x,np.array(QT)))
#plt.title('u(0, x2)')
#plt.show()

#### 7) Заново смоделировать исходные данные (см. лаб. работу № 1), увеличив мощность случайной помехи до 50…70 % от мощности полезного сигнала и провести оценку параметров. Повторить пункты 3, 4 с новыми данными.

In [274]:
#Мощность помехи — 70%

In [275]:
QT, sigma2, F, X = l2(f,Q,u,0.7)
print(QT, sigma2, F)
X.shape
#Модель неадекватная т.к. F > 1.23

7.272768548995075
[[1.23513807]
 [1.22989301]
 [0.83240737]
 [9.06425869]
 [0.21029098]
 [1.61556003]] 8.573083055815802 1.1787922299549047


(6, 100)

In [276]:
#Проверка гипотезы о незначимости каждого параметра
F=[]
for j in range(len(QT)):
    F.append(float(QT[j][0])**2/(djj[j]))
    if F[j]<3.92:
        print('Принимается')
    else:
        print('Отвергается')
print(F)

Отвергается
Отвергается
Отвергается
Отвергается
Принимается
Отвергается
[37.47696009827412, 46.162672940516515, 5.9498663617126955, 780.681595193857, 1.6464753647916515, 28.19456324372012]


In [277]:
#Проверка гипотезы о значимости самой регрессии
RSS=(np.array(df.y)-X.T.dot(QT).T)
RSS=float(RSS.dot(RSS.T))
RSSh=0
for i in range(len(df.y)):
    RSSh+=(df.y[i]-np.mean(df.u))**2
m=6
n=100
q=m-1
F=((RSSh-RSS)/q)/(RSS/(n-m))
print(F)

20.790313462322676


In [278]:
#F=1.34 < F(5,94) = 2.2899
# гипотеза о незначимости регрессии принимается

# Лаба 4

### Дисперсия возмущений – возрастающая функция от двух факторов (взвешенная сумма квадратов факторов)

#### Провести моделирование регрессионного процесса с гетероскедастичным возмущением

#Дисперсия в центре 5-10% от мощности сигнала, на краях на порядок превышала дисперсию в центре 

In [279]:
f = lambda x1, x2 : np.array([1, x1, x1**2, x2**2, x2]) 
Q = np.array([1, 1, 0.01, 10, 0.01])
u = lambda x1, x2: x1 + 0.01 * x1**2 + 0.01 * x2 + 10*x2**2 + 1
u1 = lambda x1, x2, Q: Q[0] + Q[1]*x1 + Q[2] * x1**2 + Q[3]*x2**2 + Q[4] * x2
sig2 = lambda x1,x2 : (x1**2+x2**2) 

In [280]:
z = np.array([1, sig2])


#### Полученные данные проверить по тестам на наличие гетероскедастичности

#### 1)Тест Бреуша-Пагана

In [281]:
QT, sigma2, F, X = l2(f,Q,u,0.1)

1.0390001351127587


In [282]:
et = []
for i in range(len(df.y)):
    et.append(df.y[i] - u1(df.x1[i],df.x2[i],QT))
et=np.array(et)


In [283]:
sigma2 = float(sum(et**2))/len(et) #ОМП оценка дисперсии

In [284]:
c=et/sigma2

In [285]:
ck=[]
for i in range(len(et)):
    ck.append(sig2(df.x1[i],df.x2[i]))

In [286]:
ESS = 0
c_mean = c.mean()
for i in range(len(ck)):
    ESS+=(ck[i]-c_mean)**2

In [287]:
ESS/2

33.25792389595705

In [288]:
#hi^2(0.05,1)=3.841 < ESS/2 = 25.84 => гипотеза о гомоскедастичности отвергается

#### 2)  Тест Голдфельда-Квандтона

In [289]:
l=[]
ind=[]
for i in range(len(df.x1)):
    l.append(sig2(df.x1[i],df.x2[i]))
    ind.append(i)

In [290]:
df2 = df.copy(deep=True)

In [291]:
df2['Test'] = l

In [292]:
df2 = df2.sort_values(by='Test')

In [293]:
df2.index = np.arange(len(df2))

In [294]:
df2 = df2.drop(np.r_[33:66+1])

In [295]:
df2.index = np.arange(len(df2))

In [296]:

X1 = []
for i in range(len(Q)):
    X1.append([])
    for j in range(len(df2['u'][0:33])):
        X1[i].append(f(df2['x1'][j],df2['x2'][j])[i])
X1 = np.matrix(X1)

QT = np.linalg.inv(X1.dot(X1.T))
QT = QT.dot(X1)
QT = QT.dot(np.matrix(df2.y[0:33]).T)

RSS1=(np.array(df2.y[0:33])-X1.T.dot(QT).T)
RSS1=float(RSS1.dot(RSS1.T))
print(RSS1)


36.87289075051705


In [297]:
X2 = []
for i in range(len(Q)):
    X2.append([])
    for j in range(len(df2['u'][33:66])):
        X2[i].append(f(df2['x1'][j],df2['x2'][j])[i])
X2 = np.matrix(X2)

QT = np.linalg.inv(X2.dot(X2.T))
QT = QT.dot(X2)
QT = QT.dot(np.matrix(df2.y[33:66]).T)

RSS2=(np.array(df2.y[33:66])-X2.T.dot(QT).T)
RSS2=float(RSS2.dot(RSS2.T))
print(RSS2)



303.5971352017284


In [298]:
RSS2/RSS1

8.23361361212156

In [299]:
#F(0.05,(100-33-2*5)/2,(100-33-2*5)/2) = 1.9 (вроде бы) => RSS2/RSS1 > F => гипотеза о гомоскедастичности отвергается

#### Провести оценивание параметров регрессионной модели по доступному обобщенному МНК и по обыкновенному МНК

In [300]:
QT, s, F, X = l2(f,Q,u,0.1)

1.0390001351127587


In [301]:
Q

array([ 1.  ,  1.  ,  0.01, 10.  ,  0.01])

In [302]:
QT

matrix([[9.65501075e-01],
        [1.08242778e+00],
        [3.71319167e-02],
        [9.90716021e+00],
        [2.85447524e-03]])

In [303]:
#ОМНК

In [304]:
vv = []
for i in range(len(df.x1)):
    vv.append(sig2(df.x1[i],df.x2[i])**-1)
V=np.matrix(np.diag(vv))

In [305]:
Q2 = X.dot(np.linalg.inv(V))
Q2 = np.linalg.inv(Q2.dot(X.T))
Q2 = Q2.dot(X.dot( np.linalg.inv(V)))
Q2 = Q2.dot(np.matrix(df.y).T)


In [306]:
print(Q2)

[[ 0.61229801]
 [ 1.01066667]
 [ 0.46855619]
 [10.25518046]
 [ 0.0972597 ]]


#### Сравнить эффективность оценок в этих двух случаях по квадрату их расстояния до известных истинных значений параметров

In [307]:
(np.matrix(Q).T-QT).T.dot(np.matrix(Q).T-QT)

matrix([[0.01739094]])

In [308]:
(np.matrix(Q).T-Q2).T.dot(np.matrix(Q).T-Q2)

matrix([[0.43343171]])